# Orbital-Optimization for reducing the one-norm

Author : Dr. Saad Yalouz - MSc Emiel Koridon (Cool titles)

A starting point to minimize the One-norm using Orbital-Optimization and a gradient free optimizer

## Theory 
The idea here is to employ a unitary transformation to modify the MOs of the system in order to minimize the One-norm. To build up this unitary, we start by the definition of the Orbital rotation operator which reads
$$ U(\vec{\kappa}) =\sum_{t>u}^{active} \kappa_{tu} (\hat{E}_{tu} - \hat{E}_{ut})$$
with $\hat{E}_{tu}=\frac{1} {\sqrt{2}}\sum_{\sigma}a^\dagger_{t,\sigma} a_{u,\sigma}$ and $\kappa_{tu}$ a rotation parameter affecting MOs $t$ and $u$ (note the absence of self rotation here with $t\neq u$). This operator takes the form of single excitations (with $t,u$ indices running only in the active space). Starting from this definition, a little bit of aglebra can show that implementing this rotation operator on creation/anihilation operators is strictly equivalent to redefining the MO coeff matrix of the Hamiltonian as follows
$$ \mathbf{C}_{new}(\vec{\kappa}) = \mathbf{C}_{ref} \mathbf{U}_{OO}(\vec{\kappa}) $$ 
Here, $\mathbf{U}_{OO}(\vec{\kappa})$ is a matrix representation of the effect of the rotations operator on the MOs. This matrix is an exponentiated version of the skew-matrix noted $\mathbf{K}$ encoding the parameter $\vec{\kappa}$. In other words
$$\mathbf{U}_{OO}(\vec{\kappa}) = e^{-\mathbf{K}} \text{,  with  } \mathbf{K} = skew(\vec{\kappa}) = \pmatrix{ 0 & \kappa_{01} & \kappa_{02} & \ldots & \kappa_{0N} \\
-\kappa_{01} & 0 & \kappa_{12} & \ldots & \kappa_{1N} \\
-\kappa_{02} & -\kappa_{12} & 0 &\ldots & \vdots \\
\vdots & \vdots & \vdots &\ddots & \vdots \\
-\kappa_{0N} & -\kappa_{1N} & \ldots & \ldots & 0 }$$

## Numerical implementation

Starting from this, we can encode this exponential numerically and use the kappa parameters (total number of parameters = $N(N-1)/2$) as degrees of freedom to reduce the one-norm. For this, first I use scimpy symbols to encode the matrix.

In [23]:
# import psi4
import sys
import os
import openfermion
from openfermionpyscf import run_pyscf, compute_integrals
import numpy as np
import copy
from scipy.optimize import minimize
import math as m
import scipy 
import sympy
from sympy.matrices import zeros
from random import random
from openfermion import ops, InteractionOperator, get_sparse_operator, jordan_wigner
from openfermion.functionals import JW1norm_spat
from pyscf import gto, scf, lo, tools, fci, ao2mo, mcscf, df
import module as md
import time

# Set molecule parameters.
basis = ['sto-3g', 'cc-pvdz','def2svp'][0]
multiplicity = 1

# Set calculation parameters.
optimize_occ = 1 # Do you want to also optimize the external occupied orbitals
consider_cas = 1 # Do we consider an active space or the full space?
# Set size of active space
n_orbitals = 6
n_electrons = 6
# Choose whether you want to start from localized orbitals or canonical orbitals
localize = 0
# Choose whether you want to start with a random initial rotation
randomize = 0
# Import .xyz geometry. This is kind of an involved code because I have many different geometries available...
# If you want to consider a Hydrogen chain or ring, set the following to 1
H_chain = 0
H_ring = 0
run_fci = 0
run_casci = 0
if H_chain:
    N_atoms = 12
    r = 1.8
    if consider_cas:
        description = 'H' + str(N_atoms) + str(basis) + 'ne' + str(n_electrons) +\
            'no' + str(n_orbitals)
    else:
        description = 'H' + str(N_atoms) + str(basis)
    geometry = [('H',( 0., 0., z*r)) for z in range(N_atoms)]
    
elif H_ring:
    theta = 88 * np.pi/180
    r = 1.3
    # Introduction of the molecular structure (.txt file)
    geometry = [
                ('H', (r*np.cos(theta/2.),   r*np.sin(theta/2.),  0.)),
                ('H', (r*np.cos(theta/2.),   -r*np.sin(theta/2.), 0.)),
                ('H', (-r*np.cos(theta/2.),  r*np.sin(theta/2.),  0.)),
                ('H', (-r*np.cos(theta/2.),  -r*np.sin(theta/2.), 0.))
                ]
    if consider_cas:
        description = 'H4' + str(basis) + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = 'H4' + str(basis)

else:
    fname = ['xyz_files/H2nosym.txt','xyz_files/H2COnosym.txt','xyz_files/H10.txt',\
             'xyz_files/C2.txt', 'xyz_files/LiH.txt', 'xyz_files/HLiO.txt', \
             'xyz_files/H2Onosym.txt', 'xyz_files/H14.txt', \
             'xyz_files/hnch2_s0min_dzp.txt', 'xyz_files/hnc3h6_s0min_dzp.txt',\
             'xyz_files/hnc5h10_s0min_dzp.txt', 'xyz_files/hnc7h14_s0min_dzp.txt',\
             'xyz_files/benzene.txt','xyz_files/PCy3.txt','xyz_files/PCy3Cl2Ru.txt',\
             'xyz_files/femoco.txt', 'xyz_files/M06-L.txt','xyz_files/butane.txt',\
             'xyz_files/pentane.txt','xyz_files/hexane.txt','xyz_files/heptane.txt',\
             'xyz_files/butene.txt','xyz_files/pentene.txt','xyz_files/hexene.txt',\
             'xyz_files/heptene.txt'][8]
    geometry = md.xyz_to_geom(fname)
    if consider_cas:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)\
            + 'ne' + str(n_electrons) + 'no' + str(n_orbitals)
    else:
        description = fname.replace('xyz_files/','').replace('.txt','') + str(basis)

# Determine number of AOs and electrons
nao = md.count_ao(geometry,basis,spin=multiplicity-1)
nelec = md.count_elec(geometry,basis,spin=multiplicity-1)
nmo = nao
print('nao:',nao,'\nnelec:', nelec)
threshold = 1e-10

# Set active indices
if consider_cas:
    ncore = (nelec - n_electrons) // 2
    ntot = ncore + n_orbitals # Number of core orbitals + number of active orbitals
    active_indices = list(range(ncore,ntot))
    occupied_indices = list(range(ncore))
else:
    ncore = 0
    ntot = nmo
    active_indices = list(range(nmo))
    occupied_indices = []

print(active_indices,occupied_indices)
# Build molecule and run RHF
mol = gto.Mole()
mol.atom = geometry
mol.basis = basis
mol.spin = multiplicity - 1
mol.symmetry = 0
mol.build()

if multiplicity == 1:
    myhf = scf.RHF(mol)
else:
    myhf = scf.ROHF(mol)

myhf.run()

# Extract MO_coeff and integrals. Then determine qubit 1-norm for CMOs and Pipek-Mezey MOs.

C_nonloc = np.copy(myhf.mo_coeff)

constant = float(mol.energy_nuc())


print('---------NON-LOCALIZED_ORBITALS---------')

C_copy = np.copy(C_nonloc)
t8 = time.time()
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_copy[:,:ntot], threshold)
print("calculating integrals took",time.time()-t8)

if consider_cas:
    t10 = time.time()
    CASconstant, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
    print("transforming integrals to active space ints took", time.time()-t10)
else:
    CASconstant = 0
t4 = time.time()
qub1norm = JW1norm_spat(np.copy(constant+CASconstant),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('\ncalculating norm of qubit hamiltonian took', time.time()-t4)

print('\n')

print('---------LOCALIZED_ORBITALS---------')
localizemethod = ['Pipek-Mezey','Boys','ibo','ER'][1]

C = np.copy(C_nonloc)
if localizemethod == 'Pipek-Mezey':
    orb = lo.PipekMezey(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'Boys':
    orb = lo.Boys(mol).kernel(C[:,ncore:ntot])
elif localizemethod == 'ibo':
    orb = lo.ibo.ibo(mol, C[:,ncore:ntot])
elif localizemethod == 'ER':
    orb = lo.EdmistonRuedenberg(mol).kernel(C[:,ncore:ntot])
else:
    raise ValueError('Localization method not recognized')

C_locPM = np.hstack((C_nonloc[:,:ncore],orb,C_nonloc[:,ntot:nmo]))
one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM[:,:ntot], threshold)
if consider_cas:
    CASconstant_PM, one_body_integrals, two_body_integrals =\
    md.get_active_space_integrals(one_body_integrals,
                                  two_body_integrals,
                                  occupied_indices,
                                  active_indices)
else:
    CASconstant_PM = 0
t4 = time.time()
qub1norm_locPM = JW1norm_spat(np.copy(constant+CASconstant_PM),
                        np.copy(one_body_integrals),
                        np.copy(two_body_integrals))
print('calculating norm of qubit hamiltonian took', time.time()-t4)

if consider_cas:
    print("Considering 1-norm of active space Hamiltonian")
else:
    print("Considering 1-norm of full space Hamiltonian")
    
print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)


OPT_OO_MAX_ITER  = 5000 # Maximum number of steps for the OO

# Building the K matrix : generator for the MO rotations 
Rot_param_values = []
Rot_param_names  = []
bounds = (-1.,1.)

K_mat_sym = zeros( nmo )
bounds = []
for q in range(nmo-1):
    for p in range(q+1,nmo):
        if ( p in active_indices and q in active_indices ) or\
        ( (p in occupied_indices and q in occupied_indices) and optimize_occ ):
            K_mat_sym[ p, q ] = - sympy.Symbol( "K_{}{}".format( p, q ) )
            K_mat_sym[ q, p ] = - K_mat_sym[ p, q ] 
            Rot_param_names  += [sympy.Symbol( "K_{}{}".format( p, q ))]
            Rot_param_values += [1e-1 * (random()-0.5) if randomize else 0.] # <== Set starting Rot_param_values here.
            bounds += [ [-1., 1.] ] 

# Building the bounds for the rotational parameter amplitudes in the form of constraints
# (simply beacause cobyla doesn't have intrinsic bound options)
cons = []
for factor in range(len(bounds)):
    lower, upper = bounds[factor]
    l = {'type': 'ineq',
         'fun': lambda x, lb=lower, i=factor: x[i] - lb}
    u = {'type': 'ineq',
         'fun': lambda x, ub=upper, i=factor: ub - x[i]}
    cons.append(l)
    cons.append(u)


nao: 13 
nelec: 16
[5, 6, 7, 8, 9, 10] [0, 1, 2, 3, 4]
converged SCF energy = -92.822218544079
---------NON-LOCALIZED_ORBITALS---------
calculating integrals took 0.0973215103149414
transforming integrals to active space ints took 0.0003197193145751953

calculating norm of qubit hamiltonian took 0.0016045570373535156


---------LOCALIZED_ORBITALS---------
calculating norm of qubit hamiltonian took 0.0015523433685302734
Considering 1-norm of active space Hamiltonian
starting 1-norm nonloc is: 103.23360824939041
1norm locPM is: 103.51562999277299


In [24]:
K_mat_sym

Matrix([
[    0,  K_10,  K_20,  K_30, K_40,      0,      0,      0,      0,      0,     0, 0, 0],
[-K_10,     0,  K_21,  K_31, K_41,      0,      0,      0,      0,      0,     0, 0, 0],
[-K_20, -K_21,     0,  K_32, K_42,      0,      0,      0,      0,      0,     0, 0, 0],
[-K_30, -K_31, -K_32,     0, K_43,      0,      0,      0,      0,      0,     0, 0, 0],
[-K_40, -K_41, -K_42, -K_43,    0,      0,      0,      0,      0,      0,     0, 0, 0],
[    0,     0,     0,     0,    0,      0,   K_65,   K_75,   K_85,   K_95, K_105, 0, 0],
[    0,     0,     0,     0,    0,  -K_65,      0,   K_76,   K_86,   K_96, K_106, 0, 0],
[    0,     0,     0,     0,    0,  -K_75,  -K_76,      0,   K_87,   K_97, K_107, 0, 0],
[    0,     0,     0,     0,    0,  -K_85,  -K_86,  -K_87,      0,   K_98, K_108, 0, 0],
[    0,     0,     0,     0,    0,  -K_95,  -K_96,  -K_97,  -K_98,      0, K_109, 0, 0],
[    0,     0,     0,     0,    0, -K_105, -K_106, -K_107, -K_108, -K_109,     0, 0, 0],
[    0,     

In [25]:
print(CASconstant, CASconstant_PM, constant)
print(active_indices)
print(occupied_indices)
print(ncore,ntot)
print(Rot_param_values)
print("number of opt params:", len(Rot_param_values))

-116.44552918407888 -116.44552918407888 32.98927457746205
[5, 6, 7, 8, 9, 10]
[0, 1, 2, 3, 4]
5 11
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
number of opt params: 25


In [26]:
K      = np.array( K_mat_sym.subs(list(zip(Rot_param_names,Rot_param_values))) ).astype(np.float64)  
U_OO   = scipy.linalg.expm( - K )
# print(U_OO)
print("check if U_OO is unitary:",np.allclose(np.eye(U_OO.shape[0]), U_OO @ U_OO.T))

check if U_OO is unitary: True


Then I define a cost function to encode the evaluation of the One-Norm using the new MO coeff matrix. In the latter, we compute the 1-norm from the integrals. To proceed, we use $\mathbf{C}_{new}(\vec{\kappa})$ to build up the new electronic integrals.

In [27]:
def K_matr(Rot_param_values, active_indices):
    K = zeros( nmo )
    n = 0
    for q in range(nmo-1):
        for p in range(q+1,nmo):
            if ( p in active_indices and q in active_indices ) or\
            ( (p in occupied_indices and q in occupied_indices) and optimize_occ ):
                K[ p, q ] = - Rot_param_values[n]
                K[ q, p ] = - K[ p, q ]
                n += 1
    return np.array(K)

def Cost_function_OO_OneNorm(Rot_param_values):
    """
    Cost function to minimize the One-Norm using MO rotations.
    """
    t6 = time.time()
    t5 = time.time()
    K = K_matr(Rot_param_values, active_indices)
    print("\nInitializing K-matrix took:",time.time()-t5)
    t1 = time.time()
    U_OO   = scipy.linalg.expm( - K )
    print("exponentiating matrix took:",time.time()-t1)
    t3 = time.time()
    if localize:
        C_MO   = C_locPM @ U_OO
    else:
        C_MO   = C_nonloc @ U_OO
    print("Matrix multiplication took:",time.time()-t3)
    t2 = time.time()
    one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_MO[:,:ntot], threshold)
    print("extracting integrals took:",time.time()-t2)
    t4 = time.time()
    if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
    else:
        CASconstant = 0
    t4 = time.time()
    OneNorm = JW1norm_spat(np.copy(constant+CASconstant),
                           np.copy(one_body_integrals),
                           np.copy(two_body_integrals))
    print('calculating norm of qubit hamiltonian took', time.time()-t4)
    print('1-norm is:',OneNorm)
    print("total time for 1 cost function evaluation is", time.time()-t6)
    return OneNorm

print("starting 1-norm nonloc is:",qub1norm)
print("1norm locPM is:",qub1norm_locPM)
t7 = time.time()
f_min_OO = minimize( Cost_function_OO_OneNorm,
                      x0      = Rot_param_values,
                      constraints=cons,
                      method  = 'cobyla',
                      options = {'maxiter': OPT_OO_MAX_ITER,
                                'tol'    : 1e-5,
                                'disp': True}  )
print("total time for minimization with",OPT_OO_MAX_ITER,"max iterations was:", time.time()-t7)

starting 1-norm nonloc is: 103.23360824939041
1norm locPM is: 103.51562999277299

Initializing K-matrix took: 0.02171635627746582
exponentiating matrix took: 0.003963947296142578
Matrix multiplication took: 6.127357482910156e-05
extracting integrals took: 0.10869455337524414
calculating norm of qubit hamiltonian took 0.007268190383911133
1-norm is: 103.23360824939041
total time for 1 cost function evaluation is 0.1459512710571289

Initializing K-matrix took: 0.01819467544555664
exponentiating matrix took: 0.002935171127319336
Matrix multiplication took: 5.316734313964844e-05
extracting integrals took: 0.10774540901184082
calculating norm of qubit hamiltonian took 0.006896257400512695
1-norm is: 103.23360824939043
total time for 1 cost function evaluation is 0.14246797561645508

Initializing K-matrix took: 0.02127552032470703
exponentiating matrix took: 0.002857685089111328
Matrix multiplication took: 5.316734313964844e-05
extracting integrals took: 0.11359691619873047
calculating norm 

total time for 1 cost function evaluation is 0.17730140686035156

Initializing K-matrix took: 0.04218864440917969
exponentiating matrix took: 0.0047113895416259766
Matrix multiplication took: 6.580352783203125e-05
extracting integrals took: 0.1462414264678955
calculating norm of qubit hamiltonian took 0.006858110427856445
1-norm is: 104.59074766591526
total time for 1 cost function evaluation is 0.23996543884277344

Initializing K-matrix took: 0.019684791564941406
exponentiating matrix took: 0.003134012222290039
Matrix multiplication took: 6.437301635742188e-05
extracting integrals took: 0.12017297744750977
calculating norm of qubit hamiltonian took 0.018610239028930664
1-norm is: 102.63149870589687
total time for 1 cost function evaluation is 0.18695950508117676

Initializing K-matrix took: 0.01727437973022461
exponentiating matrix took: 0.003921985626220703
Matrix multiplication took: 5.53131103515625e-05
extracting integrals took: 0.13471508026123047
calculating norm of qubit hamilt

extracting integrals took: 0.11525654792785645
calculating norm of qubit hamiltonian took 0.007095813751220703
1-norm is: 103.54195946080338
total time for 1 cost function evaluation is 0.23837828636169434

Initializing K-matrix took: 0.025434494018554688
exponentiating matrix took: 0.004538059234619141
Matrix multiplication took: 5.7220458984375e-05
extracting integrals took: 0.12290263175964355
calculating norm of qubit hamiltonian took 0.007462978363037109
1-norm is: 102.3034544238724
total time for 1 cost function evaluation is 0.1856534481048584

Initializing K-matrix took: 0.03378748893737793
exponentiating matrix took: 0.007227182388305664
Matrix multiplication took: 6.365776062011719e-05
extracting integrals took: 0.1226341724395752
calculating norm of qubit hamiltonian took 0.014118671417236328
1-norm is: 104.02485797380197
total time for 1 cost function evaluation is 0.1948838233947754

Initializing K-matrix took: 0.0608370304107666
exponentiating matrix took: 0.0099673271179

Initializing K-matrix took: 0.025193214416503906
exponentiating matrix took: 0.003841400146484375
Matrix multiplication took: 6.031990051269531e-05
extracting integrals took: 0.18018007278442383
calculating norm of qubit hamiltonian took 0.007819414138793945
1-norm is: 102.53562721412695
total time for 1 cost function evaluation is 0.2368640899658203

Initializing K-matrix took: 0.021105527877807617
exponentiating matrix took: 0.0047376155853271484
Matrix multiplication took: 7.152557373046875e-05
extracting integrals took: 0.16459441184997559
calculating norm of qubit hamiltonian took 0.00871586799621582
1-norm is: 102.24595755332214
total time for 1 cost function evaluation is 0.2209477424621582

Initializing K-matrix took: 0.03531980514526367
exponentiating matrix took: 0.00966644287109375
Matrix multiplication took: 7.915496826171875e-05
extracting integrals took: 0.15911269187927246
calculating norm of qubit hamiltonian took 0.013979673385620117
1-norm is: 102.35247808273233
total


Initializing K-matrix took: 0.05127739906311035
exponentiating matrix took: 0.02323126792907715
Matrix multiplication took: 5.14984130859375e-05
extracting integrals took: 0.1723465919494629
calculating norm of qubit hamiltonian took 0.006936311721801758
1-norm is: 102.24625826788852
total time for 1 cost function evaluation is 0.26160454750061035

Initializing K-matrix took: 0.024112939834594727
exponentiating matrix took: 0.003977060317993164
Matrix multiplication took: 5.817413330078125e-05
extracting integrals took: 0.13808894157409668
calculating norm of qubit hamiltonian took 0.008414268493652344
1-norm is: 102.38771583510045
total time for 1 cost function evaluation is 0.19552946090698242

Initializing K-matrix took: 0.07431983947753906
exponentiating matrix took: 0.018122434616088867
Matrix multiplication took: 5.173683166503906e-05
extracting integrals took: 0.12049293518066406
calculating norm of qubit hamiltonian took 0.007047414779663086
1-norm is: 102.33867782975817
total

...

extracting integrals took: 0.06183028221130371
calculating norm of qubit hamiltonian took 0.0014338493347167969
1-norm is: 102.07529844691796
total time for 1 cost function evaluation is 0.06880354881286621

Initializing K-matrix took: 0.0034143924713134766
exponentiating matrix took: 0.0008406639099121094
Matrix multiplication took: 1.2874603271484375e-05
extracting integrals took: 0.029960155487060547
calculating norm of qubit hamiltonian took 0.001428365707397461
1-norm is: 102.07528503059702
total time for 1 cost function evaluation is 0.03616476058959961

Initializing K-matrix took: 0.0031816959381103516
exponentiating matrix took: 0.0008223056793212891
Matrix multiplication took: 1.1920928955078125e-05
extracting integrals took: 0.0305173397064209
calculating norm of qubit hamiltonian took 0.0014204978942871094
1-norm is: 102.0752987632212
total time for 1 cost function evaluation is 0.03649163246154785

Initializing K-matrix took: 0.003191709518432617
exponentiating matrix took:

extracting integrals took: 0.026213407516479492
calculating norm of qubit hamiltonian took 0.003187417984008789
1-norm is: 102.07523866604272
total time for 1 cost function evaluation is 0.035982370376586914

Initializing K-matrix took: 0.0052759647369384766
exponentiating matrix took: 0.0013611316680908203
Matrix multiplication took: 3.075599670410156e-05
extracting integrals took: 0.05086231231689453
calculating norm of qubit hamiltonian took 0.0014128684997558594
1-norm is: 102.07520554505402
total time for 1 cost function evaluation is 0.06035423278808594

Initializing K-matrix took: 0.0032286643981933594
exponentiating matrix took: 0.0008196830749511719
Matrix multiplication took: 1.33514404296875e-05
extracting integrals took: 0.04653573036193848
calculating norm of qubit hamiltonian took 0.0013759136199951172
1-norm is: 102.07521853074871
total time for 1 cost function evaluation is 0.05306887626647949

Initializing K-matrix took: 0.003162860870361328
exponentiating matrix took:

extracting integrals took: 0.15143322944641113
calculating norm of qubit hamiltonian took 0.006994485855102539
1-norm is: 102.07512082974425
total time for 1 cost function evaluation is 0.24143171310424805

Initializing K-matrix took: 0.026123046875
exponentiating matrix took: 0.0038368701934814453
Matrix multiplication took: 5.4836273193359375e-05
extracting integrals took: 0.16169476509094238
calculating norm of qubit hamiltonian took 0.09313464164733887
1-norm is: 102.07510843957124
total time for 1 cost function evaluation is 0.3201298713684082

Initializing K-matrix took: 0.024122238159179688
exponentiating matrix took: 0.004923343658447266
Matrix multiplication took: 4.863739013671875e-05
extracting integrals took: 0.12750482559204102
calculating norm of qubit hamiltonian took 0.006826162338256836
1-norm is: 102.0751084991531
total time for 1 cost function evaluation is 0.17142534255981445

Initializing K-matrix took: 0.025261640548706055
exponentiating matrix took: 0.00393605232

extracting integrals took: 0.032892704010009766
calculating norm of qubit hamiltonian took 0.0013501644134521484
1-norm is: 102.07492884684005
total time for 1 cost function evaluation is 0.03902626037597656

Initializing K-matrix took: 0.003166675567626953
exponentiating matrix took: 0.0008237361907958984
Matrix multiplication took: 1.239776611328125e-05
extracting integrals took: 0.03998899459838867
calculating norm of qubit hamiltonian took 0.0015687942504882812
1-norm is: 102.07494711506875
total time for 1 cost function evaluation is 0.04675579071044922

Initializing K-matrix took: 0.0032689571380615234
exponentiating matrix took: 0.0008516311645507812
Matrix multiplication took: 1.4781951904296875e-05
extracting integrals took: 0.03165602684020996
calculating norm of qubit hamiltonian took 0.0014262199401855469
1-norm is: 102.07491715232591
total time for 1 cost function evaluation is 0.038314104080200195

Initializing K-matrix took: 0.0032014846801757812
exponentiating matrix to

extracting integrals took: 0.048891544342041016
calculating norm of qubit hamiltonian took 0.0030226707458496094
1-norm is: 102.07487473645509
total time for 1 cost function evaluation is 0.05762505531311035

Initializing K-matrix took: 0.00485682487487793
exponentiating matrix took: 0.0013837814331054688
Matrix multiplication took: 3.7670135498046875e-05
extracting integrals took: 0.05664849281311035
calculating norm of qubit hamiltonian took 0.0014448165893554688
1-norm is: 102.07485433231018
total time for 1 cost function evaluation is 0.07294368743896484

Initializing K-matrix took: 0.0032024383544921875
exponentiating matrix took: 0.0008382797241210938
Matrix multiplication took: 1.3589859008789062e-05
extracting integrals took: 0.04827237129211426
calculating norm of qubit hamiltonian took 0.0013685226440429688
1-norm is: 102.07487358394505
total time for 1 cost function evaluation is 0.05570816993713379

Initializing K-matrix took: 0.0031392574310302734
exponentiating matrix too

In [28]:
print("message:",f_min_OO.message,"number of function evaluations:",f_min_OO.nfev)

message: Optimization terminated successfully. number of function evaluations: 858


In [29]:
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_OO[:,:ntot], threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbOO = JW1norm_spat(np.copy(constant+CASconstant),
                            np.copy(one_body_integrals),
                            np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_nonloc[:,:ntot], threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorbnonloc = JW1norm_spat(np.copy(constant+CASconstant),
                                np.copy(one_body_integrals),
                                np.copy(two_body_integrals))

one_body_integrals, two_body_integrals = compute_integrals(
        mol, myhf, C_locPM[:,:ntot], threshold)
if consider_cas:
        CASconstant, one_body_integrals, two_body_integrals =\
        md.get_active_space_integrals(one_body_integrals,
                                      two_body_integrals,
                                      occupied_indices,
                                      active_indices)
else:
        CASconstant = 0
OneNormorblocPM = JW1norm_spat(np.copy(constant+CASconstant),
                       np.copy(one_body_integrals),
                       np.copy(two_body_integrals))
if randomize:
    print("Starting from random orbital rotation...")
if consider_cas:
    print("Considering CAS(" + str(n_electrons) + "," + str(n_orbitals) + "), \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)
else:
    print("Considering full space, \n1norm of CMO orb is",
         OneNormorbnonloc, "\n1norm of PM orb is", OneNormorblocPM, "\n1norm of optimizer is", OneNormorbOO)
print("number of function evaluations:",f_min_OO.nfev)

Considering CAS(6,6), 
1norm of CMO orb is 103.23360824939041 
1norm of PM orb is 103.51562999277299 
1norm of optimizer is 102.07485166091786
number of function evaluations: 858


## Visualize
If you want to save the orbitals to gaussian CUBE files

In [ ]:
Cost_function_OO_OneNorm(f_min_OO.x)
K = K_matr(f_min_OO.x, active_indices)
U_OO   = scipy.linalg.expm( - K )
if localize:
    C_OO = C_locPM @ U_OO
else:
    C_OO = C_nonloc @ U_OO
t13 = time.time()
for i in range(ncore,ntot):
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + 'onenorm_orb' + localizemethod + str(localize)\
                          + str(randomize) + str(consider_cas) + str(i) , C_OO[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)

In [ ]:
t13 = time.time()
for i in range(ncore,ntot):
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + localizemethod + str(0)\
                          + str(randomize) + str(consider_cas) + str(i) , C_nonloc[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)

In [ ]:
t13 = time.time()
for i in range(ncore,ntot):
    tools.cubegen.orbital(mol, os.getcwd() + '/CUBE_FILES/pyscfcube'\
                          + description + localizemethod + str(1)\
                          + str(randomize) + str(consider_cas) + str(i) , C_locPM[:,i])
print('Cube files of molecule', description,'created in', os.getcwd() + '/CUBE_FILES/')
print('extracting cube files took', time.time()-t13)

In [ ]:
from openfermionpyscf import run_pyscf
from openfermion import MolecularData
from openfermion.transforms import jordan_wigner
molecule = MolecularData(
    geometry, basis, multiplicity,
    description=description)
t0 = time.time()
localize_sep = 0
do_svd = 0
also_h1e = 0
do_cholesky = 0
Orth_AO = 0
run_scf = 1
run_mp2 = 0
run_cisd = 0
run_ccsd = 0
run_fci = 0
run_casci = 0
# delete_input = True
# delete_output = True
save_tohdf5 = 0
max_memory = 8000
verbose = 1
visualize = 0
localize = 1
localize_virt = 1
localize_cas = 1
only_cas = 0
# Run pyscf.
molecule = run_pyscf(molecule,
                      threshold=threshold,
                      also_h1e=also_h1e,
                      run_scf=run_scf,
                      run_mp2=run_mp2,
                      run_cisd=run_cisd,
                      run_ccsd=run_ccsd,
                      run_fci=run_fci,
                      run_casci=run_casci,
                      n_electrons=n_electrons,
                      n_orbitals=n_orbitals,
                      verbose=verbose,
                      localize=localize,
                      visualize=visualize,
                      localizemethod=localizemethod,
                      localize_virt=localize_virt,
                      localize_cas=localize_cas,
                      only_cas=only_cas,
                      localize_sep=localize_sep,
                      do_cholesky=do_cholesky,
                      do_svd=do_svd,
                      Orth_AO=Orth_AO,
                      save_tohdf5=save_tohdf5,
                      max_memory=max_memory)
print('run_pyscf took', time.time() - t0)
molecular_hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices,active_indices)
print(jordan_wigner(molecular_hamiltonian).induced_norm())